In [31]:
import itertools
import collections
from functools import lru_cache

Needed for the recursive functions, as the `lru_cache` requires all the function's arguments be hashable. 

In [32]:
Element = collections.namedtuple('Element', ['weight', 'value'])

In [2]:
def value_of(elements):
    return sum(e['value'] for e in elements)
    
def weight_of(elements):
    return sum(e['weight'] for e in elements)

# Part 1: as many bags as possible

A dynamic programming solution. See the code in part 2 (below) for an explanation: it makes more sense when we're counting the value of the bags separately from the number of bags.

In [3]:
def dp_count(elements, weight_limit):
    count_table = {(0, j): 0 for j in range(weight_limit+1)}
    back_refs = {}

    for i, element in enumerate(elements):
        for remaining_weight in range(weight_limit+1):
            if element['weight'] > remaining_weight:
                count_table[i+1, remaining_weight] = count_table[i, remaining_weight]
                back_refs[i+1, remaining_weight] = (i, remaining_weight)
            else:
                count_table[i+1, remaining_weight] = max(
                    count_table[i, remaining_weight],
                    count_table[i, remaining_weight - element['weight']] + 1)
                if count_table[i, remaining_weight] > count_table[i, remaining_weight - element['weight']] + 1:
                    back_refs[i+1, remaining_weight] = (i, remaining_weight)
                else:
                    back_refs[i+1, remaining_weight] = (i, remaining_weight - element['weight'])

    return count_table[len(elements), weight_limit], count_table, back_refs

A recursive solution. Note the use of `lru_cache`, part of the standard `functools` Python library. The `lru_cache` memoises the function it's applied to, making these recursive algorithms feasible on larger instances.

In [37]:
@lru_cache(maxsize=None)
def recursive_count(elements, weight_limit):
    if len(elements) == 0:
        return []
    else:
        this_element = list(elements)[0]
        other_elements = elements.difference(frozenset([this_element]))
        if this_element.weight > weight_limit:
            return recursive_count(other_elements, weight_limit)
        else:
            with_this = recursive_count(other_elements, weight_limit - this_element.weight)
            without_this = recursive_count(other_elements, weight_limit)
            if len(with_this) + 1 > len(without_this):
                return [this_element] + with_this
            else:
                return without_this

Finally, a greedy version that just packs the lightest bags in first, continuing while there's space for another.

In [10]:
def greedy_fill(elements, weight_limit):
    return len(list(itertools.takewhile(lambda s: s < weight_limit, itertools.accumulate(sorted(e['weight'] for e in elements)))))

# Part 2: getting the most value
First, the dynamic programming solution.

The table is indexed as `(number of bags, weight used)` and contains the solution (i.e. value of the chosen bags) for the problem when using the first few bags and a smaller weight limit. When considering adding this bag, look at the solutions for whether to use this bag or not.

Let's say we're looking at including bag _i_ and a total weight limit of _wl_. 
* If we're not using this bag, the best solution is given by the best solution for the _i_-1 other bags with the same weight limit _wl_. 
* If we _are_ using this bag, the best solution is the value of this bag plus the best solution for the other _i_-1 bags packed into (_wl_ - this bag's weight). We need the smaller weight limit to ensure there's enough space to fit this bag.

We can fill the table row by row, looking up the values in the row above. 

The final solution is read off from the table, in the cell that represents all the bags and the full weight limit.

The backpointers are there to easily trace back which decision was made when filling in each cell. That allows you to reconstruct the full solution.

In [5]:
def dp_value(elements, weight_limit):
    value_table = {(0, j): 0 for j in range(weight_limit+1)}
    back_refs = {}
    
    for i, element in enumerate(elements):
        for wl in range(weight_limit+1):
            if element['weight'] > wl:
                value_table[i+1, wl] = value_table[i, wl]
                back_refs[i+1, wl] = (i, wl)

            else:
                value_table[i+1, wl] = max(
                    value_table[i, wl],
                    value_table[i, wl - element['weight']] + element['value'])
                if value_table[i, wl] > value_table[i, wl - element['weight']] + element['value']:
                    back_refs[i+1, wl] = (i, wl)
                else:
                    back_refs[i+1, wl] = (i, wl - element['weight'])

    return value_table[len(elements), weight_limit], value_table, back_refs

A recursive definition, directly applying the ideas above. Without memoisation, this just won't terminate on even trivial-sized problems. 

In [58]:
@lru_cache(maxsize=None)
def recursive_value(elements, weight_limit):
    if len(elements) == 0:
        return frozenset()
    else:
        this_element = next(iter(elements)) # pick an arbitrary element
        other_elements = elements.difference(frozenset([this_element]))
        if this_element.weight > weight_limit:
            return recursive_value(other_elements, weight_limit)
        else:
            with_this = recursive_value(other_elements, weight_limit - this_element.weight)
            without_this = recursive_value(other_elements, weight_limit)
            items_with_this = with_this.union(frozenset([this_element]))
            if sum(e.value for e in items_with_this) > sum(e.value for e in without_this):
                return items_with_this
            else:
                return without_this

A couple of greedy attempts. The first tries to pack the items by weight, always putting in the lightest items first.

The second tries to be a bit more clever and sorts the items by pennies-per-kilo, so the highest-value bags are placed first.

In [12]:
def greedy_value_w(elements, weight_limit):
    return list(itertools.takewhile(lambda es: es['weight'] < weight_limit,
                     itertools.accumulate(
                         sorted((e for e in elements), key=lambda e: e['weight']),
                         lambda es, e: {'weight': es['weight'] + e['weight'], 'value': es['value'] + e['value']}))
               )[-1]['value']

In [11]:
def greedy_value_vpw(elements, weight_limit):
    return list(itertools.takewhile(lambda es: es['weight'] < weight_limit,
                     itertools.accumulate(
                         sorted((e for e in elements), key=lambda e: e['value'] / e['weight'], reverse=True),
                         lambda es, e: {'weight': es['weight'] + e['weight'], 'value': es['value'] + e['value']}))
               )[-1]['value']

# Utilities
A couple of functions for displaying the results. Only try these on the small example in the question, as otherwise there won't be enough screen area to display the table!

In [7]:
def display_table(table, suppress_zero=True):
    def formatted_row_element(e, suppress_zero):
        if suppress_zero and e == 0:
            return '   .'
        else:
            return '{:4d}'.format(e)
    
    
    rows = max(k[0] for k in table.keys())
    columns = max(k[1] for k in table.keys())
    for r in range(rows+1):
#         print(''.join('{:4d} '.format(table[r, c]) for c in range(columns + 1)))
        print(' '.join(formatted_row_element(table[r, c], suppress_zero) for c in range(columns + 1)))

In [8]:
def backtrace(table):
    r = max(k[0] for k in table.keys())
    c = max(k[1] for k in table.keys())
    back_table = {}
    while r > 0:
        back_table[r, c] = table[r, c]
        r, c = table[r, c]
    return back_table

In [9]:
def traced_table(base, backtrace):
    return {k: base[k] if k in backtrace else 0 for k in base}

# Solving the problems
Finally, how to solve the problem!

Load the elements, convert them into something hashable for the recursive solutions.

In [13]:
elements = [{'weight': int(l.strip().split()[0]), 'value': int(l.strip().split()[1])} 
            for l in open('../../data/09-bags.txt')]
weight_limit = 5000

In [33]:
hashable_elements = frozenset(
    Element(weight=e['weight'], value=e['value']) for e in elements
    )

## Solving part 1

All the approaches find the optimal solution.

In [14]:
value, ct, br  = dp_count(elements, weight_limit)
value

15

In [15]:
greedy_fill(elements, weight_limit)

15

In [39]:
len(recursive_count(hashable_elements, weight_limit))

15

## Solving part 2

The dynamic programming version find the optimal solution.

In [16]:
value, vt, vbr = dp_value(elements, weight_limit)
value

2383

The greedy versions don't find optimal solutions, but the smarter algorithm gets pretty close.

In [17]:
greedy_value_w(elements, weight_limit)

1801

In [18]:
greedy_value_vpw(elements, weight_limit)

2300

The recursive solution also finds the optimal solution,but only thanks to memoisation.

In [60]:
recursive_value(hashable_elements, weight_limit)

frozenset({Element(weight=301, value=134),
           Element(weight=314, value=166),
           Element(weight=320, value=154),
           Element(weight=336, value=190),
           Element(weight=337, value=140),
           Element(weight=340, value=172),
           Element(weight=353, value=191),
           Element(weight=356, value=153),
           Element(weight=359, value=171),
           Element(weight=365, value=177),
           Element(weight=381, value=166),
           Element(weight=382, value=185),
           Element(weight=414, value=189),
           Element(weight=434, value=195)})

In [61]:
sum(e.value for e in recursive_value(hashable_elements, weight_limit))

2383